In [1]:
from __future__ import print_function

### THIS IS ONLY FOR HYPERPARAMETER TUNING
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

##
## BASELINE
##
from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout, Activation
from keras.callbacks import ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adagrad, adam, sgd, rmsprop
from keras.models import Sequential

import numpy as np


Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
## THIS WON'T RUN UNLESS YOU HAVE THE (HUGE) SAVED PICKLE FILES FROM THE CHARACTER MODEL
import pickle

def data():
    pkl_file = open('test_X.pkl', 'rb')
    X_test = pickle.load(pkl_file)
    pkl_file.close()
    print ("test_X loaded")

    pkl_file = open('test_y.pkl', 'rb')
    Y_test = pickle.load(pkl_file)
    pkl_file.close()
    print ("test_y loaded")

    pkl_file = open('train_X.pkl', 'rb')
    X_train = pickle.load(pkl_file)
    pkl_file.close()
    print ("train_X loaded")

    pkl_file = open('train_y.pkl', 'rb')
    Y_train = pickle.load(pkl_file)
    pkl_file.close()
    print ("train_y loaded")

    return X_train, Y_train, X_test, Y_test

In [3]:
y_weights = np.load('y_weights.npy').item()
print (y_weights)

{0: 0.4189557794750016, 1: 3.6705746738641474, 2: 2.1043681495809157, 3: 1.1554357830642878}


In [4]:
def model(X_train, Y_train, X_test, Y_test):
    
    # assemble & compile model
    model = Sequential()
    model.add(Dense(68,input_shape=(50,68,)))
    model.add(Bidirectional(SimpleRNN(units={{choice([25, 50, 100, 150])}},
                                      activation={{choice(['relu', 'tanh'])}}),
                            merge_mode={{choice(['ave', 'sum', 'mul', 'concat'])}}
                           ))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', 
                  optimizer={{choice(['rmsprop', 'adam', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam'])}}, 
                  metrics=['categorical_accuracy'])
    # train
    model.fit(X_train, Y_train,
              batch_size={{choice([25, 50, 100, 250, 500])}},
              epochs=3,
              shuffle={{choice([True, False])}},
              class_weight = {0: 0.4189557794750016, 1: 3.6705746738641474, 2: 2.1043681495809157, 3: 1.1554357830642878},
              validation_data=(X_test, Y_test),
              verbose=1)
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('\n\n') #Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

X_train, Y_train, X_test, Y_test = data()
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=20,
                                      trials=Trials(),
                                      notebook_name='Keras_BagnallCharacter_Optimize')

test_X loaded
test_y loaded
train_X loaded
train_y loaded
>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout, Activation
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau, CSVLogger
except:
    pass

try:
    from keras.optimizers import Adagrad, adam, sgd, rmsprop
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pickle
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'units': hp.choice('units', [25, 50, 100, 150]),
        'activation': hp.choice('activation', ['relu', 'tanh']),
        'merge_mode': hp.choice('merge_mode', ['ave', 'sum', 'mul

130555/130555 [==============================] - 151s - loss: 1.3886 - categorical_accuracy: 0.2264 - val_loss: 1.3860 - val_categorical_accuracy: 0.2048
Epoch 2/3
130555/130555 [==============================] - 152s - loss: 1.3843 - categorical_accuracy: 0.2362 - val_loss: 1.3882 - val_categorical_accuracy: 0.1802
Epoch 3/3
130555/130555 [==============================] - 150s - loss: 1.3817 - categorical_accuracy: 0.2293 - val_loss: 1.3793 - val_categorical_accuracy: 0.2420



Train on 130555 samples, validate on 32641 samples
Epoch 1/3
130555/130555 [==============================] - 101s - loss: 12.0890 - categorical_accuracy: 0.5960 - val_loss: 6.5004 - val_categorical_accuracy: 0.5967
Epoch 2/3
130555/130555 [==============================] - 102s - loss: 12.0886 - categorical_accuracy: 0.5967 - val_loss: 6.5004 - val_categorical_accuracy: 0.5967
Epoch 3/3
130555/130555 [==============================] - 102s - loss: 12.0886 - categorical_accuracy: 0.5967 - val_loss: 6.5004 - va

In [8]:
print(best_run)

{'shuffle': 1, 'merge_mode': 1, 'Dropout': 0.7646166765488501, 'batch_size': 1, 'units': 3, 'optimizer': 5, 'activation': 0}


In [6]:
## Shorter still better in actual model
#
# More specific test @ 3 epochs; 20 trials:
#{'shuffle': 1, 'merge_mode': 1, 'Dropout': 0.7646166765488501, 'batch_size': 1, 'units': 3, 'optimizer': 5, 'activation': 0}
            # optimizer = adamax; 
            # dropout = 0.7646166765488501; 
            # batch_size = 50; 
            # units = 150; 
            # shuffle = False; 
            # activation = relu
            # merge_mode = 'sum'
            
# Shorter test @ 3 epochs; 20 trials: 
# {'optimizer': 0, 'Dropout': 0.2525429012036986, 'batch_size': 1, 'units': 1, 'shuffle': 1, 'activation': 0}
            # optimizer = rmsprop; 
            # dropout = 0.2525429012036986; 
            # batch_size = 50; 
            # units = 50; 
            # shuffle = False; 
            # activation = relu
            
# Longer test @ 5 epochs; 50 trials:
# {'optimizer': 0, 'Dropout': 0.5422412690636627, 'batch_size': 0, 'units': 3, 'shuffle': 1, 'activation': 0}
            # optimizer = rmsprop; 
            # dropout = 0.5422412690636627; 
            # batch_size = 25; 
            # units = 100; 
            # shuffle = False; 
            # activation = relu

In [7]:
print("Evaluation of best performing model:")
print(best_model.evaluate(X_test, Y_test))

Evaluation of best performing model:
32640/32641 [============================>.] - ETA: 0s[6.5003710097824836, 0.59670353236726814]
